**System Messages**

In [1]:
# Main chatbot system
system_chatbot = 'You are a friendly assistant. You are goofy and love to tell jokes. You are humorous and concise.'

# Chatbot response
system_should_explain_joke = 'Is the user asking you to explain a joke or pun?'
system_extract_pun = '''Which words in the user message make up the pun? 
Example: 
Q: Please explain this joke: "I used to be a baker, but I couldn\'t make enough dough."
A: The pun word(s) is: dough'''
system_pun_type = '''Is the  humor in the pun based on "sound" or "meaning"?
Example:
The pun "What do you call a fake noodle? An impasta." is based on "sound", because "impasta" sounds like "imposter".
The pun "My wife told me to stop impersonating a flamingo. I had to put my foot down." is based on "meaning", because "put my foot down" is an idiom meaning to assert authority.'''

# Explain joke
system_explain_joke = 'Explain the humor in the joke or pun.'

# Initialization

Run these blocks before using chatbot

In [ ]:
path = 'Llama-3.2-3B-Instruct-Q6_K_L.gguf'

from guidance import models, guidance, gen, assistant, user, system, select, substring
from time import sleep

llama = models.LlamaCpp(path, n_ctx=4096)

In [ ]:
import nltk

nltk.download('wordnet')
nltk.download('words')
nltk.download('cmudict')

In [4]:
@guidance
def explain_joke(lm):
    with system():
        llm = lm + system_explain_joke
    with assistant():
        llm = gen('explain_joke')
        return lm + (llm['explain_joke'] or gen())

In [5]:
from nltk.corpus import wordnet as wn

@guidance
def handle_homograph(lm, pun):
    
    synsets = wn.synsets(pun)
    definitions = [syn.definition() for syn in synsets]
    
    if len(definitions) < 2:
        return lm + explain_joke()
        
    elif len(definitions) == 2:
        def1, def2 = definitions
    
    else:
        with system():
            llm = lm + 'The pun word "' + pun + '" plays on multiple meanings. Of the following dictionary definitions, which two explain the pun? \n - ' + '\n - '.join(definitions)
            
        with assistant():
            llm += 'The humor plays off of the pun "' + pun + '" having two distinct meanings in this context. First, the definition "' + select(definitions, name = 'def1') + '"'
            if 'def1' in llm:
                def1 = llm['def1']
                definitions.remove(def1)
                llm += '. Second, "' + pun + '" also has the definition: "' + select(definitions, name = 'def2')
                if 'def2' in llm:
                    def2 = llm['def2']
                else:
                    return lm + explain_joke()
            else:
                return lm + explain_joke()
        
    with assistant():
        return lm + 'The pun "' + pun + '" is a homograph. It plays off the two meanings of the word "' + pun + '": (1) ' + def1 + ' and (2) ' + def2 + '. ' + gen(stop='.')

In [7]:
from homophones import get_homophones

@guidance
def handle_homophone(lm, pun):
    
    cleansed_pun = pun.replace('-', ' ').lower()
    cleansed_pun = ''.join([c for c in cleansed_pun if c.isalpha() or c.isspace()])
    
    similar_phones = get_homophones(cleansed_pun)
    
    if len(similar_phones) == 0:
        return lm + explain_joke()
    
    elif len(similar_phones) == 1:
        with assistant():
            homophone = similar_phones[0]
                    
    else:
        with system():
            llm = lm + 'The pun word "' + pun + '" sounds like multiple word(s). Which one is the pun? \n - ' + '\n - '.join(similar_phones)
        
        with assistant():
            llm += 'The pun "' + pun + '" is a pun because it sounds like the following word(s): ' + select(similar_phones, name = 'homophone')
        homophone = llm['homophone']
        
    with assistant():
        return lm + 'The joke relies on the homophone "' + pun + '", which sounds like "' + homophone + '". This ' + gen(stop='.')

In [8]:
@guidance
def chatbot_response(lm, user_input):
    with system():
        llm = lm + system_should_explain_joke
    
    with assistant():
        llm += select(['Yes', 'No'], name='contains_pun')

    if 'contains_pun' in llm and llm['contains_pun'] == 'Yes':
        
        with system():
            llm += system_extract_pun
            
        with assistant():
            llm += 'The pun word(s) is: ' + substring(user_input, name='pun')
        
        if 'pun' in llm:
            pun = llm['pun'].strip()
            
            with system():
                llm += system_pun_type
            
            with assistant():
                llm += select(['sound', 'meaning'], name='pun_type')
            
            if 'pun_type' in llm:
                
                if llm['pun_type'] == 'sound':
                    lm += handle_homophone(pun)
                                        
                else:
                    lm += handle_homograph(pun)
                
                return lm
    
    with assistant():
        lm += gen()
    
    return lm

# Chatbot

In [9]:
with system():
    chatbot = llama + system_chatbot

while True:
    user_input = input('Talk to the chatbot: ')
    
    if user_input.lower() in ['exit', 'quit', '', 'bye']:
        break
    
    with user():
        chatbot += user_input
    
    chatbot += chatbot_response(user_input)
    
    sleep(1)